In [ ]:
### Put your NAME and EID here:

# Problem Set 06b

Make sure you have the following packages installed for Python3:

- scikit-learn
- numpy
- matplotlib
- scipy
- skimage

In [ ]:
# imports needed
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import scipy.io
from skimage import io

# setting seed, DON'T modify
np.random.seed(10)
from pylab import rcParams
rcParams['figure.figsize'] = 10, 7

## Problem 1: Image Segmentation using K-means Clustering

In this homework, you will be applying K-means clustering to obtain segmentations of images. This is based off of an assignment of CS 376: Computer Vision taught by Dr. Kristen Grauman. You can view the course [here](http://vision.cs.utexas.edu/376-spring2018/) as well as the original assignment [here](http://vision.cs.utexas.edu/376-spring2018/assignments/a2/A2-spring2018.pdf).

Because this course does not cover topics such as image filtering and textures, we will provide starter code that already does this for you using Python libraries.

### Part A.

To be able to complete this homework, you will need be able to load in some assets (provided in the .zip). Please do the following:

- Complete the **loadImages** function to load in a set of images given filenames.
    - This will contain an option to load in grayscale versions as well.
    - Make sure to normalize the color images. ([0,255] -> [0,1])
    
- In addition, use the **loadFilterBank** function to load in the image filters to be used in the assignment. Please visualize all filters using a **5 by 8 subplot**.
    - The dimensions of the filter bank will be **49 x 49 x 38**.
    
- Now familiarize yourself with the **getFilterResponses** function that extracts **pixel features** based off of image [convolutions](https://en.wikipedia.org/wiki/Digital_image_processing).
    - The filters extracted from the previous step will be used in this function. They are essentially ways to extract information from neighboring pixels across the image.
    - The output will be **n x d**, where $n$ is the number of pixels extracted and $d$ will correspond to the number of filters in the given filter bank.
    - These features will be used in the next part to segment using kmeans.
    
    
Useful modules:
    - skimage.io.imread
    - plt.subplot
    - np.reshape
    - np.random.choice


In [ ]:
def loadImages(files = ['gumballs.jpg','snake.jpg','twins.jpg'], grayscale = False):
    images = []
    for filename in files:
        # load in image, using grayscale parameter
        x = 
        if not grayscale:
            # make sure to scale to [0,1] if color image
            x = 
        images.append(x)
    return images

In [ ]:
def loadFilterBank():
    return scipy.io.loadmat('filterBank.mat')['F']

In [ ]:
'''
Inputs:
    - imageStack : list of numpy arrays of dimension N x M x 3 (where N and M can differ between each image)
    - F          : filter bank of dimension 49 x 49 x d where d = # of filters
    - subsample  : subsampling rate to only return a percentage of pixel features
Outputs:
    - features   : 2D numpy array containing the filter responses of imageStack using F
'''
def getFilterResponses(imageStack, F, subsample=0.05):
    features = []
    for image in imageStack:
        N, M, d = image.shape[0], image.shape[1], F.shape[2]
        responses = np.zeros((N,M,d))
        for i in range(d):
            filter = F[:,:,i]
            responses[:,:,i] = scipy.ndimage.convolve(image,filter)
            
        responses = responses.reshape(N*M, d)
        sample_idx = np.random.choice(N*M, int(N*M*subsample), replace=False)
        sampled_responses = responses[sample_idx,:]
        features.append(sampled_responses)
    features = np.concatenate(features, axis=0)
    return features

### Part B.

In this section, we will be using k-means to compute texture features. The features themselves will be histograms of **quantized** filter responses of a neighborhood around a pixel. We will walk you through this process with the following steps:

- First complete the **createTextons** function. 
    - This function will take an array of gathered **filterResponses** as input.
    - Now, using the KMeans module from scikit-learn, fit it on filterResponses.
    - This will return a trained KMeans (**textons**) to be used in the next function.
    
- Now, complete the **getTextonFeatures** function.
    - Most of it has been completed, but it still needs to be filled in by using the previous function.
    - It should take as input a **trained** KMeans module (**textons**) from calling createTextons in another cell. 
    - (1) The function will first convolve the input filter bank on an input image. (This is done for you)
        - The result of this will be an **N x M x d** array of responses.
    - (2) Next, create another array of size **N x M** of quantized filter responses for each pixel. To do this:
        - We could naively loop through each pixel, but this can be quickly vectorized by doing the following.
        - First, reshape the array from (1) into an array of **NM x d** array. 
        - Next, pass this into the kmeans module to compute their quantized filter responses (i.e. nearest texton). This should be a 1D numpy array containing the new labels. We are essentially mapping the filter responses into a discrete amount of vectors for (3).
        - Finally, reshape the labels back into dimension **N x M**.
    - (3) Next, we will create **histogram features** of each pixel using their neighborhood's quantized results. Initialize a new array of size **N x M x k**. For each pixel do the following:
        - Using the window_size input, look at surrounding pixels (in a box). (This is already completed with inner for loops).
        - Now collect the neighborhood's corresponding quantized labels from step (2). Specifically, collect the number of each quantized label in the neighborhood.
            - We have setup an array of size $k$ for this -- just increment each index based on the neighborhood.
        - Now store this result in the histogram array.
        - **Note:** This can also be vectorized (i.e. without the two inner for loops) but be sure to use correct bounds.
        
        
     - Now just return the array from (3).
    
Useful modules:
    - sklearn.cluster.KMeans (.predict)
    - np.reshape

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
'''
Inputs:
    - filterResponses : 2D numpy array containing a set of filter responses from pixels. 
                            - Each row should contain d filter responses where d = size of filter bank used
    - numTextons      : int, specifying the number of cluster centers to find from filterResponses
Outputs:
    - textons         : scikit-learn KMeans module that has been fitted on filterResponses
'''
def createTextons(filterResponses, numTextons=50):
    textons = # Create KMeans module
    # Now fit on filterResponses 
    
    return textons

In [ ]:
'''
Inputs:
    - image        : grayscale image as a numpy array of size N x M
    - textons      : an sklearn.cluster.KMeans module that has been trained on filterResponses
    - F            : filter bank of dimension 49 x 49 x d where d = # of filters
    - window_size  : dimension of square window centered at current pixel to collect histogram information
Outputs:
    - features   : 2D numpy array containing the filter responses of imageStack using F
'''
def getTextonFeatures(image, textons, F, window_size = 7):
    N, M, d = image.shape[0], image.shape[1], F.shape[2]
    k = textons.n_clusters
    
    print("Step 1...")
    # Step (1)
    responses = np.zeros((N,M,d))
    for i in range(d):
        filter = F[:,:,i]
        responses[:,:,i] = scipy.ndimage.convolve(image,filter)
    
    print("Step 2...")
    # Step (2)
    # add in as much code to obtain a final quantized_responses array
    quantized_responses = 
    
    print("Step 3...")
    # Step (3)
    histograms = np.zeros((N,M,k))
    for i in range(N):
        for j in range(M):
            # current histogram should have k entries where:
            #    i-th element will contain the # of times a quantized centroid occurs
            hist = np.array([0 for i in range(k)])
            
            # go through the neighborhood
            for x in range( max(0, i - window_size // 2), min(N, i + window_size // 2 + 1) ):
                for y in range( max(0, j - window_size // 2), min(M, j + window_size // 2 + 1) ):
                    current_quantized = # collect the neighborhood stats
                    # don't forget to update hist
                    
            # set the pixel's completed histogram
            histograms[i,j] = hist
    
    return histograms

### Part C.

Once the previous two functions have been written, we can now compute the image segmentations. This will be done by another K-means clustering. Please do the following:

- Complete the createSegmentation function.
    - The input will be **N x M x d** array of features. 
    - Run K-means on this (make sure to first reshape to **NM x d**) using the given k.
    - Now provide the correct label for each feature.
    - **Note:** this is pretty similar to step (2) of getTextonFeatures, except you must train another kmeans module.

Useful modules:
    - sklearn.cluster.KMeans
    - np.reshape

In [ ]:
'''
Inputs:
    - features     : 3D numpy array containing a set of features for each pixel in an image
    - k            : int, specifying the number of image segments desired
Outputs:
    - segmentation : 2D numpy array containing a label for each pixel based on features
'''
def createSegmentation(features, k=5):
    N,M,d = features.shape
    kmeans = # create kmeans module
    
    # first fit then predict label of each pixel
    
    return # return segmentation of size N*M x d

### Part D.

Now to do some visualizations! To do this, you will have to write up the whole pipeline that connects each previous part together. We have already written starter code to setup the filterBank and imageStacks from Part A. Now you must do the rest.

- (1) Collect the filterResponses from **imageStackGray**.
    - Simply call getFilterResponses and pass in imageStackGray and filter bank F.
    
- (2) Create a textons (Kmeans of responses) module using **createTextons** from data in (1).
    - We suggest using the default value of numTextons=50, but this can be experimented with.
    
Now, **for each image** in imageStack/imageStackGray, do the following (3) to (5) + Visualizations:


- (3) Take the current image (**grayscale version**) and extract the histogram features from **part C**.
    - You can also experiment with window_size to try and understand how these features change.

- (4) Create the segmentation (**part D**) of textonFeatures from (3).
    - Feel free to adjust the value of $k$ to test.

- (5) In addition, create a **color segmentation** from the original color image.
    - This can be done by simply passing in the color version of this image into createSegmentation.
    - Feel free to adjust the value of $k$
    
Now please visualize the following:

- The original color image.

- Texture Segmentation from (4).
    - Make sure to title it with the parameters used (numTextons, window_size and $k$)
    
- Color Segmentation from (5).
    - Make sure to title it with the value of $k$ used.
    

Useful modules:
    - plt.imshow

In [ ]:
imageStack = loadImages(grayscale=False)
imageStackGray = loadImages(grayscale=True)

In [ ]:
F = loadFilterBank()

In [ ]:
image = 
imageGray = 

## Turn in Instructions

Once you have completed Problem 1, please submit (for this part of the assignment):

- This .ipynb file.
- A PDF version of this file. To do this:
    1. Go to File -> Download as -> HTML
    2. Open the HTML and Print, and change the **destination** to **PDF**.